<img src="https://datasciencecampus.ons.gov.uk/wp-content/uploads/sites/10/2017/03/data-science-campus-logo-new.svg"
             alt="ONS Data Science Campus Logo"
             width = "240"
             style="margin: 0px 60px"
             />

In [11]:
# import the helper functions from the parent directory,
# these help with things like graph plotting and notebook layout
import sys
sys.path.append('../..')
from helper_functions import *

# set things like fonts etc - comes from helper_functions
set_notebook_preferences()

# add a show/hide code button - also from helper_functions
toggle_code(title = "import functions")

In [12]:
#dependencies
import pandas as pd
titanic = pd.read_csv('../../Data/titanic.csv')

#columns from NewVar script
titanic['child'] = (titanic['age'] < 18).astype(int)
titanic['embarked_city'] = titanic['embarked'].map({'S':'Southampton','C':'Cherbourg','Q':'Queenstown'})
titanic['surname'] = titanic['name'].str.split(',',expand=True)[0]

#columns from descstat script
titanic['z_score'] = (titanic['fare'] - titanic['fare'].mean())/titanic['fare'].std()


toggle_code(title = "Dependencies")

# 8. Merging Data

Often all the data you need to answer a question are not contained within a single dataset, but across several. Datasets can be joined, or 'merged', to allow data to be analysed together, but only **if the two datasets share a common reference or identifier.**

Linking data come in a number of forms, and are commonly referred to as 'indexical' data. Some examples include:

* Your NHS number, allowing data linkage across the NHS for primary, secondary, tertiary care episodes and prescribing.
* Any account number (e.g. banking, utilities, travel card, council tax etc.) can acts as a point of linkage between different sets of data.
* Your email, phone number, social media handles etc.
* Your address can also act as a spatial reference, linking you to your neighbourhood, local services etc.

## 8.1 Different ways to merge

While you may have heard it called "Join" in other languages, particularly in database query languages, in pandas we use the `.merge()` function.

Once you have established that two dataframes share a reference that will permit a merge to be executed, you may wish to further specify how the merge behaves with the `how` parameter.

* Inner - Only rows with reference values that appear in both dataframes are merged.
* Left - All the data from the 'left' dataframe is retained, and any rows that have matching references are merged from the 'right' dataframe.
* Right - All data from the right and anything that matches from the left. Effectively, the reverse of 'Left'.
* Outer (Full) - all data from the left and right dataframe is retained, matched up where possible.

This can be easier to understand graphically:

![joins](https://www.dofactory.com/Images/sql-joins.png)

Let’s read in some additional titanic data and have a look at them.

The new dataset includes the passenger name and age, as well as the additional variables:
* boat - lifeboat identifier
* body - body identification number
* home.dest - the passenger's home and destination in the form "home / dest" or just "home".

The dataset is located in the 'Data' folder, it is an excel file called: 'titanic_more.xlsx'.

In [13]:
# Read in the titanic_more.xlsx using pandas.
titanic_more = pd.read_excel('../../data/titanic_more.xlsx')
titanic_more.head()

name      age boat   body  \
0                    Allen, Miss. Elisabeth Walton  29.0000    2    NaN   
1                   Allison, Master. Hudson Trevor   0.9167   11    NaN   
2                     Allison, Miss. Helen Loraine   2.0000  NaN    NaN   
3             Allison, Mr. Hudson Joshua Creighton  30.0000  NaN  135.0   
4  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  25.0000  NaN    NaN   

                         home.dest  
0                     St Louis, MO  
1  Montreal, PQ / Chesterville, ON  
2  Montreal, PQ / Chesterville, ON  
3  Montreal, PQ / Chesterville, ON  
4  Montreal, PQ / Chesterville, ON

In order to merge the two tables we need to use a column which uniquely defines each passenger and is available in both dataframes. At first glance, 'name' would appear to be a good candidate for this, however, remember there are a couple of passengers who have the same name as each other.

**If a common code is available, then it would be advisable to merge on that in preference to any entity name, as codes tend to have a standardised format.**

We can explicitly check if a column uniquely identifies rows with the `Series.is_unique` property.

In [14]:
# Name is not unique defined for each row in titanic
titanic['name'].is_unique

False

In [15]:
# Name is not unique defined for each row in titanic_more
titanic_more['name'].is_unique

False

In this case, we can create a field that will uniquely identify passengers in both datesets by generating a new variable that combines the 'name' and 'age' variables. This is because we happen to know the ages of the passengers who have the same name.

In [16]:
# Create unique id based on name and age for titanic
titanic['name_age_id'] = titanic['name'] + " " + titanic['age'].astype(str)
# Check if the new variable is unique
titanic['name_age_id'].is_unique

True

In [17]:
# Create unique id based on name and age for titanic_more
titanic_more['name_age_id'] = titanic_more['name'] + " " + titanic_more['age'].astype(str)
# Check if the new variable is unique
titanic_more['name_age_id'].is_unique

True

## 8.2 Merge function


Now that we have unique id fields in both titanic and titanic_more, we use them to merge the two datasets.

In [18]:
# merge the titanic_more dataset with titanic
titanic_merge = titanic.merge(titanic_more[['name_age_id','boat','body','home.dest']], on = 'name_age_id')
titanic_merge.head()

pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked  child  \
0  29.0000      0      0   24160  211.3375       B5        S      0   
1   0.9167      1      2  113781  151.5500  C22 C26        S      1   
2   2.0000      1      2  113781  151.5500  C22 C26        S      1   
3  30.0000      1      2  113781  151.5500  C22 C26        S      0   
4  25.0000      1      2  113781  151.5500  C22 C26        S      0   

  embarked_city  surname   z_score  \
0   Southampton    Allen  3.439849   
1   Southampton  Allison  2.284729   
2   Southampton  Allison  2.284729   
3   Southampton  Allison  2.284729   
4   Southampton  Allison  2.284729   

                                         name_age_id boat   body  \
0                 Allen, Miss. Elisabeth Walton 29.0    2    NaN   
1              Allison, Master. Hudson Trevor 0.9167   11    NaN   
2                   Allison, Miss. Helen Loraine 2.0  NaN    NaN   
3          Allison, Mr. Hudson Joshua Creighton 30.0  NaN  135.0   
4  Allison, Mrs. Hudson J C (Bessie Waldo Daniels...  NaN    NaN   

                         home.dest  
0                     St Louis, MO  
1  Montreal, PQ / Chesterville, ON  
2  Montreal, PQ / Chesterville, ON  
3  Montreal, PQ / Chesterville, ON  
4  Montreal, PQ / Chesterville, ON

In the code cell above we merge the `titanic_more` dataframe into `titanic`. We do this on the basis of the 'name_age_id' variable that we created.

The default merge behaviour is 'inner' join, however in this particular case all behaviours ('inner', 'left', 'right','outer') resolve to the same outcome as both datasets include the same 1,309 passengers.

In [19]:
# Note that pandas actually handles multiple columns directly for unique identification.
titanic_merge = titanic.merge(titanic_more[['name','age','boat','body','home.dest']], on = ['name','age'])

# Exercise 7

Load the revised dataset 'titanic_revised.xlsx' which only includes passengers which have a value for boat and home.dest and try merging this to titanic.

1. Which types of merge perform as expected?
    * Try the different `how` parameters: 'inner', 'outer', 'left', 'right'.
2. How many values are observed for boat, body, and home.dest?
    * i.e. How many values are non-missing/
    * Hint: use `.count()`


In [20]:
## read in titanic_revised dataset
#titanic_revised = pd.read_excel('../../data/titanic_revised.xlsx')

## Create unique id field
#titanic_revised['name_age_id'] = titanic_revised['name'] + " " + titanic_revised['age'].astype(str)

## Merge the titanic datasets

##Question 1

## left is the obvious choice for this join
#titanic_merge2 = titanic.merge(titanic_revised[['name_age_id','boat','body','home.dest']], how = 'left', on = 'name_age_id')

## outer join works too
##titanic_merge2 = titanic.merge(titanic_revised[['name_age_id','boat','body','home.dest']], how = 'outer', on = 'name_age_id')
#print('Left and Outer joins work for merging these datasets.\n')

## Question 2
#print("counts of observations in 'boat', 'body', and 'home.dest'")
#print(titanic_merge2[['boat','body','home.dest']].count(),'\n')


toggle_code()

Left and Outer joins work for merging these datasets.

counts of observations in 'boat', 'body', and 'home.dest'
boat         314
body           0
home.dest    314
dtype: int64 



# Homework

Please complete the **DataMerge.ipynb** notebook found in the **homework_tasks** folder.